In [2]:
import sys
import json
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [3]:
ind = {'000000': '\\textbf{Total nonfarm}',
       '110099': 'Mining \& logging',
       '230000': 'Construction',
       '320000': 'Durable goods manufacturing',
       '340000': 'Nondurable goods manufacturing',
       '510000': 'Information',
       '510099': 'Financial activities',
       '540099': 'Professional \& business services',
       '440000': 'Retail trade',
       '480099': 'Transportation, warehousing, \& utilities',
       '610000': 'Educational services',
       '620000': 'Health care \& social services',
       '720000': 'Accommodation \& food services',
       '910000': 'Federal government',
       '920000': 'State \& local government'}

In [54]:
series = {'JTS00000000JOL': 'Openings', 
          'JTS00000000HIL': 'Hires', 
          'JTS00000000QUL': 'Quits',
          'JTS00000000TSL': 'Separations',
          'LNS13000000': 'Unemp'}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'QUR'
    series[i] = i
    
df = bls_api(series, (2000, 2020), bls_key)

df.to_csv(data_dir/'jolts_master.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [71]:
series2 = {}
for code, name in ind.items():
    i = 'JTS' + code + '00' + 'JOR'
    series2[i] = i
    
df = bls_api(series2, (2000, 2020), bls_key)

df.to_csv(data_dir/'jolts_master2.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [4]:
df = (pd.read_csv(data_dir / 'jolts_master.csv', parse_dates=['date'])
        .set_index('date'))

levels_srs = ['Openings', 'Hires', 'Quits', 'Separations', 'Unemp']
rates_srs = ['JTS72000000QUR', 'JTS00000000QUR']

levels_data = df[levels_srs].dropna() / 1000
levels_data.to_csv(data_dir / 'jolts.csv', index_label='date')

ltdata = levels_data.iloc[-1]

ltdate = dtxt(ltdata.name)['mon1']
write_txt(text_dir / 'jolts_ltdate.txt', ltdate)
ltopen = ltdata['Openings']
lthire = ltdata['Hires']
ltquit = ltdata['Quits']
ltsep = ltdata['Separations']
ltun = ltdata['Unemp']

ratio = ltdata['Openings'] / ltdata['Unemp']
ratiop = levels_data['Openings'].iloc[-12] / levels_data['Unemp'].iloc[-12]
p3date = dtxt(levels_data.index[-37])['mon1']
ratio3 = levels_data['Openings'].iloc[-37] / levels_data['Unemp'].iloc[-37]

text = (f'In {ltdate}, there were {ltopen:.1f} million total job openings '+
        f'and {lthire:.1f} million hires completed. In the same month '+
        f'there were {ltsep:.1f} million total separations, of '+
        f'which {ltquit:.1f} million were voluntary. In comparison, '+
        f'there are {ltun:.1f} million unemployed persons in {ltdate}. The ratio '+
        f'of job openings to unemployed persons was {ratio:.1f} in the latest '+
        f'month, compared to {ratiop:.1f} in the same month one year prior, and '+
        f'{ratio3:.1f} in {p3date}.')

write_txt(text_dir / 'jolts2.txt', text)

text

'In December 2019, there were 6.4 million total job openings and 5.9 million hires completed. In the same month there were 5.7 million total separations, of which 3.5 million were voluntary. In comparison, there are 5.8 million unemployed persons in December 2019. The ratio of job openings to unemployed persons was 1.1 in the latest month, compared to 1.2 in the same month one year prior, and 0.8 in December 2016.'

In [9]:
rates_data = df[rates_srs].dropna().rename({'JTS00000000QUR': 'TOT_QU', 'JTS72000000QUR': 'AFS_QU'}, axis=1)

rates_data.to_csv(data_dir / 'quits.csv', index_label='date')

node = end_node(rates_data['AFS_QU'], 'red!50!purple')
write_txt(text_dir / 'quits_afs_node.txt', node)

node = end_node(rates_data['TOT_QU'], 'violet')
write_txt(text_dir / 'quits_tot_node.txt', node)

ltdata = rates_data.iloc[-1]

afs_max = rates_data['AFS_QU'].max()
afs_idxmax = dtxt(rates_data['AFS_QU'].idxmax())['mon1']

text = (f'In {ltdate}, the total quits rate in all industries was {ltdata.TOT_QU} percent. '+
        f'The accommodations and food services quits rate was {ltdata.AFS_QU} '+
        f'percent; the series high for the industry group was {afs_max} percent in {afs_idxmax}.')

write_txt(text_dir / 'quits_afs.txt', text)

text

'In December 2019, the total quits rate in all industries was 2.3 percent. The accommodations and food services quits rate was 4.9 percent; the series high for the industry group was 6.3 percent in January 2001.'

In [10]:
max_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().min()
max_qur2 = df.loc['2015':,df.columns.str.endswith('QUR')].dropna().max()
min_qur2 = df.loc['2015':,df.columns.str.endswith('QUR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur, 
                    'maxst': max_qur2, 'minst': min_qur2})

quits = {}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'QUR'
    quits[i] = name
    
qur.index = qur.index.map(quits)
qur['width'] = qur['maxst'] - qur['minst']
qur['end'] = qur['max'].max() - qur['max']
qur.index.name = 'name'
final = qur[['min', 'width', 'latest', 'end']].sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(qur)))]
final.to_csv(data_dir / 'quits_ind.csv', sep=';')

In [30]:
df = (pd.read_csv(data_dir / 'jolts_master2.csv', parse_dates=['date'])
        .set_index('date'))

max_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().max()
min_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().min()
lt_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().iloc[-1]

jor = pd.DataFrame({'max': max_jor, 'min': min_jor, 'latest': lt_jor})

jos = {}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'JOR'
    jos[i] = name
    
jor.index = jor.index.map(jos)
jor['width'] = jor['max'] - jor['min']
jor.index.name = 'name'
final = jor[['min', 'width', 'latest']].sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.06 for i in list(range(0, len(jor)))]
final.to_csv(data_dir / 'openings_ind.csv', sep=';')

In [11]:
final

,min,width,latest,end,y
name,,,,,
Accommodation \& food services,2.6,1.6,4.9,0.0,0.08
Retail trade,1.7,1.3,3.2,2.4,-0.92
Professional \& business services,1.5,0.8,2.8,2.9,-1.92
"Transportation, warehousing, \& utilities",0.8,1.2,2.5,3.8,-2.92
\textbf{Total nonfarm},1.2,0.5,2.3,3.8,-3.92
Construction,0.7,1.4,2.2,3.0,-4.92
Health care \& social services,1.2,0.6,2.1,4.0,-5.92
Nondurable goods manufacturing,0.8,1.1,1.9,3.8,-6.92
Mining \& logging,0.6,1.6,1.7,3.3,-7.92


In [17]:
qur['end'] = qur['max'].max() - qur['max']

name
\textbf{Total nonfarm}                       3.8
Mining \& logging                            3.3
Construction                                 3.0
Durable goods manufacturing                  4.7
Nondurable goods manufacturing               3.8
Information                                  3.6
Financial activities                         4.0
Professional \& business services            2.9
Retail trade                                 2.4
Transportation, warehousing, \& utilities    3.8
Educational services                         4.5
Health care \& social services               4.0
Accommodation \& food services               0.0
Federal government                           4.9
State \& local government                    5.4
Name: max, dtype: float64

In [36]:
df

,Openings,Hires,Quits,Separations,Unemp,JTS00000000QUR,JTS11009900QUR,JTS23000000QUR,JTS32000000QUR,JTS34000000QUR,JTS51000000QUR,JTS51009900QUR,JTS54009900QUR,JTS44000000QUR,JTS48009900QUR,JTS61000000QUR,JTS62000000QUR,JTS72000000QUR,JTS91000000QUR,JTS92000000QUR
date,,,,,,,,,,,,,,,,,,,,
2000-01-01,NaN,NaN,NaN,NaN,5708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-01,NaN,NaN,NaN,NaN,5858.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-01,NaN,NaN,NaN,NaN,5733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-01,NaN,NaN,NaN,NaN,5481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-01,NaN,NaN,NaN,NaN,5758.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-01,7032.0,5951.0,3471.0,5798.0,5753.0,2.3,1.5,2.5,1.6,1.8,1.5,1.5,3.0,3.3,2.1,1.3,1.9,4.7,0.6,0.8
2019-10-01,7361.0,5782.0,3497.0,5652.0,5857.0,2.3,1.7,2.4,1.4,1.7,1.6,1.5,2.9,2.9,2.2,1.6,1.9,5.0,0.6,0.8
2019-11-01,6787.0,5827.0,3568.0,5709.0,5811.0,2.3,1.9,2.2,1.4,2.0,1.5,1.7,2.9,3.9,2.2,1.4,2.0,4.6,0.6,0.8


In [38]:
max_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().min()
max_qur2 = df.loc['2015':,df.columns.str.endswith('QUR')].dropna().max()
min_qur2 = df.loc['2015':,df.columns.str.endswith('QUR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur, 
                    'maxst': max_qur2, 'minst': min_qur2})

In [39]:
qur

,max,min,latest,maxst,minst
JTS00000000QUR,2.5,1.2,2.3,2.4,1.9
JTS11009900QUR,3.0,0.6,1.7,3.0,1.4
JTS23000000QUR,3.3,0.7,2.2,2.8,1.4
JTS32000000QUR,1.6,0.5,1.4,1.6,0.9
JTS34000000QUR,2.5,0.8,1.9,2.2,1.1
JTS51000000QUR,2.7,0.5,1.6,2.1,1.1
JTS51009900QUR,2.3,0.7,1.4,1.8,0.8
JTS54009900QUR,3.4,1.5,2.8,3.4,2.6
JTS44000000QUR,3.9,1.7,3.2,3.9,2.6
JTS48009900QUR,2.5,0.8,2.5,2.5,1.3


In [40]:
final = pd.DataFrame()

In [14]:
max_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().min()
max_qur2 = df.loc['2018':,df.columns.str.endswith('QUR')].dropna().max()
min_qur2 = df.loc['2018':,df.columns.str.endswith('QUR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur, 
                    'maxst': max_qur2, 'minst': min_qur2})

quits = {}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'QUR'
    quits[i] = name
    
final = pd.DataFrame()
final['outer1'] = qur['min']
final['range1'] = qur['minst'] - qur['min']
final['rangest'] = qur['maxst'] - qur['minst']
final['range2'] = qur['max'] - qur['maxst']
final['outer2'] = qur['max'].max() - qur['max']
final['latest'] = qur['latest']
final.index = final.index.map(quits)
final = final.sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(qur)))]
final.index.name = 'name'
final.to_csv(data_dir / 'quits_ind.csv', sep=';')

In [15]:
df = (pd.read_csv(data_dir / 'jolts_master2.csv', parse_dates=['date'])
        .set_index('date'))

max_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().max()
min_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().min()
max_jor2 = df.loc['2018':,df.columns.str.endswith('JOR')].dropna().max()
min_jor2 = df.loc['2018':,df.columns.str.endswith('JOR')].dropna().min()
lt_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().iloc[-1]

jor = pd.DataFrame({'max': max_jor, 'min': min_jor, 'latest': lt_jor, 
                    'maxst': max_jor2, 'minst': min_jor2})

jos = {}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'JOR'
    jos[i] = name
    
final = pd.DataFrame()
final['outer1'] = jor['min']
final['range1'] = jor['minst'] - jor['min']
final['rangest'] = jor['maxst'] - jor['minst']
final['range2'] = jor['max'] - jor['maxst']
final['outer2'] = jor['max'].max() - jor['max']
final['latest'] = jor['latest']
final.index = final.index.map(jos)
final = final.sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(jor)))]
final.index.name = 'name'
final.to_csv(data_dir / 'openings_ind.csv', sep=';')

In [48]:
final

,outer1,range1,rangest,range2,outer2,latest,y
name,,,,,,,
Accommodation \& food services,2.6,1.1,1.6,1.0,0.0,4.9,0.06
Retail trade,1.7,0.9,1.3,0.0,2.4,3.2,-0.94
Professional \& business services,1.5,1.1,0.8,0.0,2.9,2.8,-1.94
"Transportation, warehousing, \& utilities",0.8,0.5,1.2,0.0,3.8,2.5,-2.94
\textbf{Total nonfarm},1.2,0.7,0.5,0.1,3.8,2.3,-3.94
Construction,0.7,0.7,1.4,0.5,3.0,2.2,-4.94
Health care \& social services,1.2,0.3,0.6,0.2,4.0,2.1,-5.94
Nondurable goods manufacturing,0.8,0.3,1.1,0.3,3.8,1.9,-6.94
Mining \& logging,0.6,0.8,1.6,0.0,3.3,1.7,-7.94
